# Langfuse

In [9]:
from langfuse import Langfuse
from dotenv import load_dotenv
import os
from langfuse.callback import CallbackHandler

load_dotenv("/home/kevin/projects/langgraph/.env")
langfuse = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
)
 
langfuse_handler = CallbackHandler()

In [24]:
# Tests the SDK connection with the server
langfuse_handler.auth_check()

True

# Langgraph


## TypedDict와 Dict의 차이점 예시

In [1]:
# TypedDict와 Dict의 차이점 예시
from typing import Dict, TypedDict

# 일반적인 파이썬 딕셔너리(dict) 사용
sample_dict: Dict[str, str] = {
    "name": "테디",
    "age": "30",  # 문자열로 저장 (dict 에서는 가능)
    "job": "개발자",
}


# TypedDict 사용
class Person(TypedDict):
    name: str
    age: int  # 정수형으로 명시
    job: str


typed_dict: Person = {"name": "셜리", "age": 25, "job": "디자이너"}

In [2]:
# dict의 경우
sample_dict["age"] = 35  # 문자열에서 정수로 변경되어도 오류 없음
sample_dict["new_field"] = "추가 정보"  # 새로운 필드 추가 가능

# TypedDict의 경우
typed_dict["age"] = 35  # 정수형으로 올바르게 사용
typed_dict["age"] = "35"  # 타입 체커가 오류를 감지함
typed_dict["new_field"] = (
    "추가 정보"  # 타입 체커가 정의되지 않은 키라고 오류를 발생시킴
)

In [3]:
from typing import Annotated

name: Annotated[str, "사용자 이름"]
age: Annotated[int, "사용자 나이 (0-150)"]

In [7]:
from typing import Annotated, List
from pydantic import Field, BaseModel, ValidationError


class Employee(BaseModel):
    id: Annotated[int, Field(..., description="직원 ID")]
    name: Annotated[str, Field(..., min_length=3, max_length=50, description="이름")]
    age: Annotated[int, Field(gt=18, lt=65, description="나이 (19-64세)")]
    salary: Annotated[
        float, Field(gt=0, lt=10000, description="연봉 (단위: 만원, 최대 10억)")
    ]
    skills: Annotated[
        List[str], Field(min_items=1, max_items=10, description="보유 기술 (1-10개)")
    ]


# 유효한 데이터로 인스턴스 생성
try:
    valid_employee = Employee(
        id=1, name="테디노트", age=30, salary=1000, skills=["Python", "LangChain"]
    )
    print("유효한 직원 데이터:", valid_employee)
except ValidationError as e:
    print("유효성 검사 오류:", e)

# 유효하지 않은 데이터로 인스턴스 생성 시도
try:
    invalid_employee = Employee(
        name="상현",  # 이름이 너무 짧음
        age=17,  # 나이가 범위를 벗어남
        salary=20000,  # 급여가 범위를 벗어남
        skills="Python",  # 리스트가 아님
    )
except ValidationError as e:
    print("유효성 검사 오류:")
    for error in e.errors():
        print(f"- {error['loc'][0]}: {error['msg']}")

유효한 직원 데이터: id=1 name='테디노트' age=30 salary=1000.0 skills=['Python', 'LangChain']
유효성 검사 오류:
- id: Field required
- name: String should have at least 3 characters
- age: Input should be greater than 18
- salary: Input should be less than 10000
- skills: Input should be a valid list


## Reducer 예시
-add_messages
`messages` 키는 [`add_messages`](https://langchain-ai.github.io/langgraph/reference/graphs/?h=add+messages#add_messages) 리듀서 함수로 주석이 달려 있으며, 이는 LangGraph에게 기존 목록에 새 메시지를 추가하도록 지시합니다. 

주석이 없는 상태 키는 각 업데이트에 의해 덮어쓰여져 가장 최근의 값이 저장됩니다. 

`add_messages` 함수는 2개의 인자(left, right)를 받으며 좌, 우 메시지를 병합하는 방식으로 동작합니다.

In [1]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

In [2]:
class GraphState(TypedDict):
    question: Annotated[list[str], add_messages]
    context: Annotated[str, "Context"]
    answer: Annotated[str, "Answer"]
    messages: Annotated[list[str], add_messages]
    relevance: Annotated[str, "Relevance"]

In [7]:
state: GraphState = {
    "question": ["반가워요"],
    "context": "",
    "answer": "",
    "messages": [],
    "relevance": ""
}

In [9]:
state['question']

['반가워요']

In [8]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

# 기본 사용 예시
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [AIMessage(content="반갑습니다~", id="2")]

result1 = add_messages(msgs1, msgs2)
print(result1)

[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='1'), AIMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='2')]


In [9]:
# 동일한 ID를 가진 메시지 대체 예시
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [HumanMessage(content="반갑습니다~", id="1")]

result2 = add_messages(msgs1, msgs2)
print(result2)

[HumanMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='1')]


## 간단한 챗봇

In [10]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class GraphState(TypedDict):
    messages: Annotated[list[str], add_messages]

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

def chatbot(state: GraphState) -> GraphState:
    return {"messages": [llm.invoke(state["messages"])]}

In [13]:
graph_builder = StateGraph(GraphState)
graph_builder.add_node("chatbot", chatbot)

In [16]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [17]:
graph = graph_builder.compile()

In [18]:
question = "서울의 유명한 맛집 TOP 10 추천해줘"

# 그래프 이벤트 스트리밍
for event in graph.stream({"messages": [("user", question)]}):
    # 이벤트 값 출력
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: 서울에는 맛있는 음식점이 정말 많습니다! 다음은 서울에서 유명한 맛집 TOP 10을 추천해 드립니다.

1. **광화문 미진** - 전통적인 비빔밥과 한정식으로 유명한 곳입니다. 깔끔한 분위기에서 다양한 한식을 즐길 수 있습니다.

2. **남산돈까스** - 바삭한 돈까스와 풍부한 소스를 자랑하는 곳으로, 많은 사람들에게 사랑받는 맛집입니다.

3. **신사동 가로수길의 이태원 클라쓰** - 다양한 한국식 치킨과 맥주를 즐길 수 있는 인기 있는 치킨집입니다.

4. **골목식당** - 다양한 길거리 음식을 맛볼 수 있는 곳으로, 특히 떡볶이와 순대가 유명합니다.

5. **서래마을의 파리바게뜨** - 다양한 베이커리와 디저트를 제공하는 곳으로, 프랑스식 빵과 케이크가 인기입니다.

6. **을지로의 유림** - 전통적인 한우를 사용한 고기집으로, 질 좋은 고기를 즐길 수 있는 곳입니다.

7. **청담동의 스시효** - 신선한 재료를 사용한 일식 스시 전문점으로, 미쉐린 스타를 받은 곳입니다.

8. **마포의 홍대 돈부리** - 일본식 덮밥 요리로 유명한 곳으로, 다양한 메뉴가 제공됩니다.

9. **종로의 명동교자** - 칼국수와 만두로 유명한 곳으로, 특히 칼국수가 인기입니다.

10. **강남의 배틀트립** - 다양한 한국 요리를 한자리에서 즐길 수 있는 뷔페식당으로, 다양한 메뉴가 준비되어 있습니다.

각 맛집마다 독특한 매력이 있으니, 방문하실 때 참고하시면 좋을 것 같습니다!


In [30]:
for event in graph.stream({"messages": [("user", question)]}, config={"callbacks":[langfuse_handler]}):
    print(event)

{'chatbot': {'messages': [AIMessage(content='서울에는 다양한 맛집이 많이 있습니다. 아래는 서울에서 유명한 맛집 TOP 10을 추천해 드립니다. 각 음식 종류에 따라 다르게 선정해보았습니다.\n\n1. **부암동 미술관** - 한정식: 전통 한정식을 현대적으로 재해석한 곳으로, 아름다운 경치와 함께 정갈한 한식을 즐길 수 있습니다.\n\n2. **마포 갈매기** - 갈매기살: 마포구에 위치한 이곳은 신선한 갈매기살과 다양한 반찬이 인기입니다.\n\n3. **광화문 미진** - 설렁탕: 깊고 진한 국물 맛이 일품인 설렁탕 전문점으로, 많은 사람들이 찾는 곳입니다.\n\n4. **이태원 앤드류스** - 브런치: 다양한 브런치 메뉴와 아늑한 분위기로 유명한 카페입니다.\n\n5. **종로 통인시장** - 전통 간식: 다양한 전통 먹거리를 경험할 수 있는 시장으로, 특히 찹쌀떡과 호떡이 유명합니다.\n\n6. **삼청동 수연산방** - 전통찻집: 고즈넉한 분위기에서 전통 차와 함께 다과를 즐길 수 있는 곳입니다.\n\n7. **홍대 명랑핫도그** - 핫도그: 독특한 종류의 핫도그와 다양한 소스를 제공하는 인기 있는 길거리 음식점입니다.\n\n8. **신사동 가로수길** - 디저트 카페: 다양한 디저트를 즐길 수 있는 카페들이 밀집해 있어, 특히 케이크와 마카롱이 유명합니다.\n\n9. **여의도 이화수전통육개장** - 육개장: 진한 국물과 푸짐한 고기가 매력적인 육개장 전문점입니다.\n\n10. **강남 뚝배기집** - 비빔밥: 다양한 재료로 만든 비빔밥이 인기이며, 건강한 한 끼를 원하시는 분들께 추천합니다.\n\n각 맛집은 지역에 따라 다르니 방문하시기 전에 위치와 운영 시간을 확인하시는 것이 좋습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 484, 'prompt_tokens': 19, 'total